# Action PixelBytes: Catching Insights in Unified Multimodal Sequences

## Description

**Action-PixelBytes** est un modèle conçu pour générer simultanément du texte, des images, des animations pixel par pixel et des actions-états sous forme de séquences. L'objectif de ce projet est d'explorer un embedding unifié qui permet une génération multimodale cohérente, facilitant ainsi l'interaction entre différentes formes de données.

## Dataset

Pour ce projet, nous utilisons le dataset **PixelBytes-PokemonSprites**. Contrairement à la version précédente, **PixelBytes-Pokemon**, cette version est structurée à la volée pour un format unifié. Cela signifie que les données sont préparées de manière à être directement utilisées pour l'entraînement du modèle, ce qui simplifie le processus d'embedding.

## Tokenizer

Le tokenizer joue un rôle dans la préparation des données pour le modèle. Voici comment il fonctionne pour chaque type de donnée :

### Traitement du Texte
Le texte est d'abord normalisé et converti en minuscules. Ensuite, il est encodé en format ASCII pour garantir que tous les caractères sont traités de manière uniforme. Cette étape permet de simplifier le texte avant de le transformer en une séquence de tokens.

### Traitement des Images
Les images, y compris les GIFs, sont traitées en plusieurs étapes. Chaque image est convertie en un espace colorimétrique LAB, qui est plus adapté à certaines analyses d'image. Ensuite, chaque frame d'une image animée est quantifiée selon une palette de couleurs prédéfinie, ce qui permet de réduire la complexité des données tout en préservant les informations essentielles.

### Traitement des Actions-États
Les actions-états sont normalisées pour assurer que toutes les valeurs sont sur la même échelle. Cela facilite la comparaison et l'analyse des états d'action. Les états sont ensuite quantifiés selon un ensemble prédéfini d'états d'action, ce qui permet au modèle de mieux comprendre les relations entre différentes actions.

### Création de Séquences
Les séquences sont créées en utilisant un contexte spatial et temporel. Cela signifie que pour chaque séquence, le modèle prend en compte non seulement l'entrée actuelle, mais aussi les entrées précédentes. Cela permet de générer des entrées de plusieurs éléments qui contiennent des informations pertinentes pour la tâche à accomplir.

## État du Projet

Le code est encore en cours de développement. Bien que les principales fonctionnalités du tokenizer et du traitement des données soient implémentées, des améliorations et des optimisations sont à venir. L'objectif est de rendre le modèle plus robuste et efficace pour la génération multimodale.

## Prochaines Étapes

- Finaliser l'architecture du modèle.
- Implémenter l'entraînement et l'évaluation.
- Optimiser les performances du modèle.
- Effectuer des tests approfondis sur différents types de données multimodales.


In [1]:
#!pip install -q mamba-ssm causal-conv1d ## for GPU (Mambapy included)
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [2]:
# only in kaggle for HF
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")
# no warning msg during train
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
# our approach
from pixelbytes import *

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

def count_parameters_in_k(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000

In [4]:
from datasets import load_dataset
hf_dataset = load_dataset("ffurfaro/PixelBytes-PokemonAll")['train'].train_test_split(test_size=0.1, seed=42)
train_ds, val_ds = hf_dataset['train'], hf_dataset['test']

Generating train split:   0%|          | 0/533 [00:00<?, ? examples/s]

In [8]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_REDUCTION = 2
tokenizer = ActionPixelBytesTokenizer(data_slicing=DATA_REDUCTION)
# Paramètres
VOCAB_SIZE = tokenizer.vocab_size
EMBED_SIZE = 128
HIDDEN_SIZE = 512
NUM_LAYERS = 2
PXBY_DIM = 6 # tokenizer
AR = True
MODEL_TYPE = "lstm"
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ACCUMULATION_STEPS = 4
SEQ_LENGTH = 1024
STRIDE = 512

config = ModelConfig(vocab_size=VOCAB_SIZE, embed_size=EMBED_SIZE, hidden_size=HIDDEN_SIZE, 
                          num_layers=NUM_LAYERS, pxby_dim=PXBY_DIM, auto_regressive=AR, model_type=MODEL_TYPE)
config

ModelConfig {
  "AR": true,
  "d_conv": 4,
  "d_state": 512,
  "embed_size": 126,
  "expand": 2,
  "hidden_size": 512,
  "num_layers": 2,
  "pxby_dim": 6,
  "pxby_emb": 21,
  "transformers_version": "4.44.0",
  "vocab_size": 151
}

In [9]:
# Initialisation du modèle
model = aPxBySequenceModel(config).to(DEVICE)
print(f"Le modèle a {count_parameters_in_k(model):.2f}k paramètres entraînables.")
# Parametre d'entrainement
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=-100)
scaler = GradScaler() if torch.cuda.is_available() else None

Le modèle a 3879.92k paramètres entraînables.


In [ ]:
# Préparation des données
def dataloading(ds):
    dataset = TokenPxByDataset(ds, tokenizer, SEQ_LENGTH, STRIDE)
    sampler = ShuffledSampler(dataset, seed=42)
    return DataLoader(dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn, sampler=sampler)
train_dataloader, val_dataloader = dataloading(train_ds), dataloading(val_ds)

In [ ]:
# Entraînement
model.train_model(train_dataloader, val_dataloader, optimizer, criterion, DEVICE, scaler, EPOCHS, ACCUMULATION_STEPS)

Evaluating: 100%|██████████| 107/107 [00:12<00:00,  8.86it/s]


Validation Loss: 5.0196, Validation Accuracy: 0.0031


Training:  29%|██▉       | 297/1030 [00:39<01:36,  7.58it/s]

In [17]:
import re
from huggingface_hub import HfApi, create_repo, whoami
def push_model_to_hub(repo_name, model_dir, token, subfolder=None):
    api = HfApi(token=token)
    subfolder = re.sub(r'[^a-zA-Z0-9]+', '_', subfolder).strip('_').lower()

    try:
        create_repo(repo_name, token=token, repo_type="model", exist_ok=True)
        username = whoami(token=token)['name']
        repo_id = f"{username}/{repo_name}"
        print(f"Repository '{repo_id}' created or already exists.")
    except Exception as e:
        print(f"Error creating repository: {e}")
        return
    
    api.upload_folder(
        folder_path=model_dir,
        repo_id=repo_id,
        repo_type="model",
        path_in_repo=subfolder,
        ignore_patterns=[".*"],  # Ignorer les fichiers cachés
        create_pr=False  # Créer directement dans la branche principale
    )
    print(f"Model pushed successfully to {repo_name}, subfolder: {subfolder}")
!ls

lstm_predictive_best  lstm_predictive_last  training_metrics.csv


In [16]:
# save model
push_model_to_hub("aPixelBytes-PokemonLSTM", "lstm_predictive_last", hf_token, "lstm_predictive_last")

Repository 'ffurfaro/aPixelBytes-PokemonLSTM' created or already exists.


model.safetensors:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

Model pushed successfully to aPixelBytes-PokemonLSTM, subfolder: lstm_predictive_last


In [ ]:
# Test de génération
test_input = next(iter(dataloader))['input_ids'][:1].to(DEVICE)
generated = model.generate(test_input, max_length=100)
print("Generated sequence:", generated)